Prompt Optimization

Prompt Engineering recommends many best practices to build a prompt that completes the best output for the targetted goal.
It is an iterative process.

This notebook started from codes in Antrophic Building with the Calude API course.

The first class definition is to abstract away available LLM solutions.

In [6]:
import json
import concurrent.futures
import re
from textwrap import dedent
from statistics import mean
from dotenv import load_dotenv
from anthropic import Anthropic

class LLMClient:
    model = None

    def __init__(self):
        model=None

    def getModel(self):
        return model

In [7]:
from anthropic import Anthropic

class AntrophicClient(LLMClient):
    client = None
    def __init__(self, model):
        LLMClient.__init__(self)
        self.model = model
        client = Anthropic()()


    def add_user_message(messages, text):
        user_message = {"role": "user", "content": text}
        messages.append(user_message)


    def add_assistant_message(messages, text):
        assistant_message = {"role": "assistant", "content": text}
        messages.append(assistant_message)


    def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
        params = {
            "model": model,
            "max_tokens": 1000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }

        if system:
            params["system"] = system

        message = client.messages.create(**params)
        return message.content[0].text